<a href="https://colab.research.google.com/github/ArunAravind2001/Named-Entity-Recognition-Model/blob/main/NER_MOdel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import minibatch


In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_excel('Labelled_dataset.xlsx')


In [ ]:
# Load a pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Create a DocBin object to store training examples
doc_bin = DocBin()

# Iterate over the rows of the DataFrame to create training examples
for _, row in df.iterrows():
    text = row['Article_Text']
    spans = []

    # Extract entity information from other columns
    for col in df.columns:
        if col not in ['Article_No', 'Article_Text']:
            start = text.find(str(row[col]))
            end = start + len(str(row[col]))
            spans.append(Span(doc=nlp.make_doc(text), start=start, end=end, label=col))

    doc = nlp.make_doc(text)
    for span in spans:
        doc.spans[span.label] = doc.spans.get(span.label, []) + [span]
    example = Example.from_dict(doc, {})
    doc_bin.add(doc)

# Save the training data to disk
doc_bin.to_disk("training_data.spacy")

NameError: name 'Span' is not defined

In [ ]:
import spacy
from spacy.tokens import DocBin, Span
from spacy.training import Example
import pandas as pd

# Load a pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")  # You can also use larger models

# Create a DocBin object to store training examples
doc_bin = DocBin()

# Load your dataset
df = pd.read_excel('labelled_dataset.xlsx')

# Iterate over the rows of the DataFrame to create training examples
for _, row in df.iterrows():
    text = row['Article_Text']
    doc = nlp.make_doc(text)  # Create the Doc object once
    spans = []  # List to store spans

    # Extract entity information from other columns
    for col in df.columns:
        if col not in ['Article_No', 'Article_Text']:
            value = str(row[col])
            start = text.find(value)  # Find the start index
            if start != -1:  # Ensure the value was found
                end = start + len(value)  # Calculate the end index
                if end <= len(doc):  # Check if the end index is valid
                    spans.append(Span(doc, start, end, label=str(col)))  # Create the Span

    # Create a list of entity tuples for the Example
    entity_tuples = [(span.start, span.end, span.label_) for span in spans]

    # Check alignment with spaCy's built-in function
    # If misalignment warnings are still shown, it might be necessary to ensure that spans
    # correspond correctly with actual text. This part can be skipped if alignment is confirmed.
    try:
        example = Example.from_dict(doc, {"entities": entity_tuples})
        doc_bin.add(example.reference)  # Add the reference doc to the DocBin
    except Exception as e:
        print(f"Error with text: {text}, {e}")

# Save the training data to disk
doc_bin.to_disk("training_data.spacy")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Garmin(NYSE: GRMN) acquired Lumishore, a Swansea, ..." with entities "[(28, 37, 'Company_Receiving_Investment'), (0, 6, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [ ]:
print(df.columns)


Index(['Article_No', 'Company_Receiving_Investment', 'Investment_Amount',
       'Investing_Companies', 'Round_of_Investment', 'Location_of_the_Company',
       'Date', 'CEO_Name', 'Article_Text'],
      dtype='object')


In [ ]:
import spacy
from spacy.tokens import DocBin, Span
from spacy.training import Example
import pandas as pd
import random

# Load a pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")  # You can also use larger models

# Create a DocBin object to store training examples
doc_bin = DocBin()

# Load your dataset
df = pd.read_excel('labelled_dataset.xlsx')

# Create a blank NER pipeline if it doesn't exist
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe("ner")

# Add your new labels to the NER model from the DataFrame columns
for col in df.columns:
    if col not in ['Article_No', 'Article_Text']:
        ner.add_label(col)  # Add labels from relevant columns

# Prepare training data
training_data = []

for _, row in df.iterrows():
    text = row['Article_Text']
    entities = []

    # Create the entities from the other columns
    for col in df.columns:
        if col not in ['Article_No', 'Article_Text']:
            value = str(row[col])
            start = text.find(value)
            if start != -1:
                end = start + len(value)
                entities.append((start, end, col))  # Create a tuple for the entity

    training_data.append((text, {"entities": entities}))  # Append the text and entities to the training data

# Train the model
optimizer = nlp.resume_training()
for i in range(10):  # Number of iterations
    losses = {}
    # Shuffle the training data
    random.shuffle(training_data)
    # Create minibatches
    for batch in spacy.util.minibatch(training_data, size=8):  # Size can be adjusted
        for text, annotations in batch:
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update([example], drop=0.5, losses=losses)
    print(f"Iteration {i} - Losses: {losses}")

# Save the trained model
nlp.to_disk("trained_ner_model")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Garmin(NYSE: GRMN) acquired Lumishore, a Swansea, ..." with entities "[(28, 37, 'Company_Receiving_Investment'), (0, 6, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Hong Kong, Singapore, London, and Dubai, Global, O..." with

Iteration 0 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 306.31614880721605}
Iteration 1 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 94.53651996785375}
Iteration 2 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 80.62561745109119}
Iteration 3 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 72.04632275520727}
Iteration 4 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 69.91087591655074}
Iteration 5 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 65.73261862847896}
Iteration 6 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 363.8939645022619}
Iteration 7 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 105.08298369521728}
Iteration 8 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 178.78157152307375}
Iteration 9 - Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 64.32698021825931}


In [ ]:
# Load the trained model
nlp_trained = spacy.load("trained_ner_model")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
# Load the trained NER model
nlp_trained = spacy.load("trained_ner_model")

# The test article
test_article = """
With Nothing Underneath, a London, UK-based womenswear brand, raised £2.5M in funding. The round was led by Pembroke VCT alongside JamJar Investments. The company intends to use the funds to accelerate the expansion of its team and of its UK retail presence, grow its presence in overseas markets in the U.S. and Europe – specifically in the DACH region (Germany, Austria and Switzerland), and further develop new product lines. Founded in 2017 by Pip Durell, former Tatler editor and Vogue stylist, With Nothing Underneath provides shirts and other wardrobe staples with inspiration drawn from icons like Carolyn Bessette and Charlotte Rampling. Their sourcing and practices are recognised by a B Corp certification. The business has already generated an international customer base and received organic endorsements from the likes of Dolly Alderton, Cara Delevingne and Florence Pugh. It has also collaborated with well-known brand, Free People, and has a limited-edition partnership with the Financial Times. FinSMEs 21/10/2024
"""

# Process the article through the trained model
doc = nlp_trained(test_article)

# Function to extract information in a structured format
def extract_information(doc):
    extracted_info = {
        "Company_Receiving_Investment": "N/A",
        "Investment_Amount": "N/A",
        "Investing_Companies": "N/A",
        "Round_of_Investment": "N/A",
        "Location_of_the_Company": "N/A",
        "Date": "N/A",
        "CEO_Name": "N/A"
    }

    for ent in doc.ents:
        if ent.label_ == "Company_Receiving_Investment":
            extracted_info["Company_Receiving_Investment"] = ent.text
        elif ent.label_ == "Investment_Amount":
            extracted_info["Investment_Amount"] = ent.text
        elif ent.label_ == "Investing_Companies":
            extracted_info["Investing_Companies"] = ent.text
        elif ent.label_ == "Round_of_Investment":
            extracted_info["Round_of_Investment"] = ent.text
        elif ent.label_ == "Location_of_the_Company":
            extracted_info["Location_of_the_Company"] = ent.text
        elif ent.label_ == "Date":
            extracted_info["Date"] = ent.text
        elif ent.label_ == "CEO_Name":
            extracted_info["CEO_Name"] = ent.text

    return extracted_info

# Extract information from the test article
info = extract_information(doc)

# Display the extracted information
print(f"Extracted Information: {info}")


Extracted Information: {'Company_Receiving_Investment': 'N/A', 'Investment_Amount': 'N/A', 'Investing_Companies': 'N/A', 'Round_of_Investment': 'N/A', 'Location_of_the_Company': 'London, UK', 'Date': 'N/A', 'CEO_Name': 'N/A'}


In [ ]:
# Import necessary libraries
import spacy
from spacy.tokens import DocBin, Span
from spacy.training import Example
import pandas as pd
import random

# Load a pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")  # You can also use larger models like "en_core_web_md"

# Load your labeled dataset
df = pd.read_excel('labelled_dataset.xlsx')

# Create a DocBin object to store training examples
doc_bin = DocBin()

# Add custom labels to the NER model
ner = nlp.get_pipe("ner")
for label in df.columns[1:]:  # Assuming the first column is 'Article_No'
    ner.add_label(label)

# Prepare training data
for _, row in df.iterrows():
    text = row['Article_Text']
    doc = nlp.make_doc(text)  # Create the Doc object once
    spans = []  # List to store spans

    # Extract entity information from other columns
    for col in df.columns[1:]:  # Skip 'Article_No' and focus on entity columns
        value = str(row[col])
        start = text.find(value)  # Find the start index
        if start != -1:  # Ensure the value was found
            end = start + len(value)  # Calculate the end index
            if end <= len(doc):  # Check if the end index is valid
                spans.append(Span(doc, start, end, label=col))  # Create the Span

    # Create a list of entity tuples for the Example
    entity_tuples = [(span.start, span.end, span.label_) for span in spans]

    # Create an example and add it to the DocBin
    example = Example.from_dict(doc, {"entities": entity_tuples})
    doc_bin.add(example.reference)  # Add the reference doc to the DocBin

# Save the training data to disk
doc_bin.to_disk("training_data.spacy")

# Prepare for training
nlp.resume_training()  # Start/resume training process
random.seed(42)

# Convert generator to list for shuffling
docs = list(doc_bin.get_docs(nlp.vocab))
random.shuffle(docs)  # Shuffle training data

# Training Loop
for i in range(10):  # Number of iterations (adjustable)
    losses = {}
    for doc in docs:  # Loop through each training example
        example = Example.from_dict(doc, {"entities": doc.ents})
        nlp.update([example], drop=0.5, losses=losses)
    print(f"Iteration {i + 1}: Losses: {losses}")

# Use the trained model on a new article
test_article = """
With Nothing Underneath, a London, UK-based womenswear brand, raised £2.5M in funding. The round was led by Pembroke VCT alongside JamJar Investments. The company intends to use the funds to accelerate the expansion of its team and of its UK retail presence, grow its presence in overseas markets in the U.S. and Europe – specifically in the DACH region (Germany, Austria and Switzerland), and further develop new product lines. Founded in 2017 by Pip Durell, former Tatler editor and Vogue stylist, With Nothing Underneath provides shirts and other wardrobe staples with inspiration drawn from icons like Carolyn Bessette and Charlotte Rampling. Their sourcing and practices are recognised by a B Corp certification. The business has already generated an international customer base and received organic endorsements from the likes of Dolly Alderton, Cara Delevingne and Florence Pugh. It has also collaborated with well-known brand, Free People, and has a limited-edition partnership with the Financial Times. FinSMEs 21/10/2024
"""

# Process the test article
doc = nlp(test_article)

# Extract entities
extracted_info = {
    "Company_Receiving_Investment": "N/A",
    "Investment_Amount": "N/A",
    "Investing_Companies": "N/A",
    "Round_of_Investment": "N/A",
    "Location_of_the_Company": "N/A",
    "Date": "N/A",
    "CEO_Name": "N/A"
}

# Populate extracted_info with values from the doc.ents
for ent in doc.ents:
    if ent.label_ == "Company_Receiving_Investment":
        extracted_info["Company_Receiving_Investment"] = ent.text
    elif ent.label_ == "Investment_Amount":
        extracted_info["Investment_Amount"] = ent.text
    elif ent.label_ == "Investing_Companies":
        if extracted_info["Investing_Companies"] == "N/A":
            extracted_info["Investing_Companies"] = ent.text
        else:
            extracted_info["Investing_Companies"] += f", {ent.text}"
    elif ent.label_ == "Location_of_the_Company":
        extracted_info["Location_of_the_Company"] = ent.text
    elif ent.label_ == "Date":
        extracted_info["Date"] = ent.text
    elif ent.label_ == "CEO_Name":
        extracted_info["CEO_Name"] = ent.text

# Print the extracted information in the desired format
print("Extracted Information:", extracted_info)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Garmin(NYSE: GRMN) acquired Lumishore, a Swansea, ..." with entities "[(28, 37, 'Company_Receiving_Investment'), (0, 6, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteration 1: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 293.1393080661691}
Iteration 2: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 117.97375602335808}
Iteration 3: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 96.18403791534183}
Iteration 4: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 85.75334678070358}
Iteration 5: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 79.81872612897276}
Iteration 6: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 76.4446043418578}
Iteration 7: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 70.95285601470798}
Iteration 8: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 68.55609506440729}
Iteration 9: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 56.58014587031191}
Iteration 10: Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 50.944618123728105}
Extracted Information: {'Company_Receiving_Investment': 'N

In [ ]:
import pandas as pd

def get_entity_positions(text, entities):
    """
    Calculate the start and end positions of each entity in the given text.

    Parameters:
    text (str): The text to search.
    entities (dict): A dictionary of entity names and their values.

    Returns:
    dict: A dictionary with the start and end positions for each entity.
    """
    positions = {}
    for entity_name, entity_value in entities.items():
        start_pos = text.find(str(entity_value))
        end_pos = start_pos + len(str(entity_value)) if start_pos != -1 else -1
        positions[f"{entity_name}_Start_Pos"] = start_pos
        positions[f"{entity_name}_End_Pos"] = end_pos
    return positions

# Load your dataset
data = pd.read_excel('labelled_dataset.xlsx')

# Iterate over each row and compute the start and end positions for each entity
for _, row in data.iterrows():
    entity_dict = {
        "Company_Receiving_Investment": row["Company_Receiving_Investment"],
        "Investment_Amount": row["Investment_Amount"],
        "Investing_Companies": row["Investing_Companies"],
        "Round_of_Investment": row["Round_of_Investment"],
        "Location_of_the_Company": row["Location_of_the_Company"],
        "Date": row["Date"],
        "CEO_Name": row["CEO_Name"]
    }
    positions = get_entity_positions(row["Article_Text"], entity_dict)
    for col, pos in positions.items():
        data.at[_, col] = pos

# Save the updated DataFrame to a new Excel file
data.to_excel("updated_labelled_dataset.xlsx", index=False)

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 804.5 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy.tokens import DocBin, Span
from spacy.training import Example
import pandas as pd
import random
from spacy.util import minibatch  # Import minibatch utility

# Load a better pre-trained spaCy model
nlp = spacy.load("en_core_web_lg")  # Use the medium model for better performance

# Create a DocBin object to store training examples
doc_bin = DocBin()

# Load your updated dataset
data = pd.read_excel('updated_labelled_dataset.xlsx')

# Iterate over the rows of the DataFrame to create training examples
for _, row in data.iterrows():
    text = row['Article_Text']
    doc = nlp.make_doc(text)  # Create the Doc object

    spans = []  # List to store spans
    # Create spans for each entity based on start and end positions
    for col in data.columns:
        if 'Start_Pos' in col:  # Check for start position columns
            start_col = col
            end_col = start_col.replace('Start_Pos', 'End_Pos')
            start = row[start_col]
            end = row[end_col]
            if start != -1 and end != -1 and end <= len(doc):  # Check for valid positions
                label = start_col.replace('_Start_Pos', '')
                if label in nlp.get_pipe("ner").labels:  # Ensure the label is in the NER model
                    spans.append(Span(doc, start, end, label=label))  # Create Span

    # Create a list of entity tuples for the Example
    entity_tuples = [(span.start, span.end, span.label_) for span in spans]

    # Create an example for training
    example = Example.from_dict(doc, {"entities": entity_tuples})

    # Add the doc to the DocBin
    doc_bin.add(doc)  # Add the actual Doc object, not the Example

# Save the training data to disk
doc_bin.to_disk("training_data.spacy")

# **Training the Model**
# Check if the NER pipeline exists; if not, create one
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)

# Add your new labels to the NER model
for label in data.columns:
    if 'Start_Pos' not in label and 'End_Pos' not in label and label != 'Article_Text':
        ner.add_label(label)

# Prepare for training
optimizer = nlp.resume_training()
random.seed(42)  # For reproducibility
losses = {}  # Dictionary to store losses

# Training loop
for iteration in range(10):  # Adjust number of iterations as necessary
    docs = list(doc_bin.get_docs(nlp.vocab))  # Convert generator to list
    random.shuffle(docs)  # Shuffle the training data

    # Create minibatches
    batches = minibatch(docs, size=8)  # Size can be adjusted
    for batch in batches:
        examples = []
        for doc in batch:
            # Extracting spans for the current document
            text = doc.text
            spans = []
            # Find the corresponding row in your dataset for this document
            row = data[data['Article_Text'] == text].iloc[0]
            for col in data.columns:
                if 'Start_Pos' in col:  # Check for start position columns
                    start_col = col
                    end_col = start_col.replace('Start_Pos', 'End_Pos')
                    start = row[start_col]
                    end = row[end_col]
                    if start != -1 and end != -1 and end <= len(doc):  # Check for valid positions
                        label = start_col.replace('_Start_Pos', '')
                        if label in nlp.get_pipe("ner").labels:  # Ensure the label is in the NER model
                            spans.append(Span(doc, start, end, label=label))  # Create Span

            if spans:
                # Create example with extracted spans
                example = Example.from_dict(doc, {"entities": [(span.start, span.end, span.label_) for span in spans]})
                examples.append(example)

        # Update the model
        nlp.update(examples, drop=0.5, losses=losses)

# Save the trained model
nlp.to_disk("trained_ner_model")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
import spacy

# Load the trained model
nlp = spacy.load("trained_ner_model")

# Sample text for testing
test_text = """With Nothing Underneath, a London, UK-based womenswear brand, raised £2.5M in funding.
The round was led by Pembroke VCT alongside JamJar Investments.
The company intends to use the funds to accelerate the expansion of its team and of its UK retail presence,
grow its presence in overseas markets in the U.S. and Europe – specifically in the DACH region (Germany, Austria and Switzerland),
and further develop new product lines. Founded in 2017 by Pip Durell, former Tatler editor and Vogue stylist,
With Nothing Underneath provides shirts and other wardrobe staples with inspiration drawn from icons like Carolyn Bessette and Charlotte Rampling.
Their sourcing and practices are recognised by a B Corp certification. The business has already generated an international customer base
and received organic endorsements from the likes of Dolly Alderton, Cara Delevingne and Florence Pugh.
It has also collaborated with well-known brand, Free People, and has a limited-edition partnership with the Financial Times.
FinSMEs 21/10/2024"""

# Process the text with the NER model
doc = nlp(test_text)

# Print the recognized entities
print("Entities found:")
for ent in doc.ents:
    print(f"{ent.text} - {ent.label_}")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Entities found:
London - GPE
UK - GPE
£2.5M - MONEY
Pembroke VCT - ORG
JamJar Investments - ORG
UK - GPE
U.S. - GPE
Europe - LOC
DACH - GPE
Germany - GPE
Austria - GPE
Switzerland - GPE
2017 - DATE
Pip Durell - PERSON
Tatler - ORG
Vogue - ORG
Carolyn Bessette - PERSON
Charlotte Rampling - PERSON
B Corp - ORG
Dolly Alderton - PERSON
Cara Delevingne - PERSON
Florence Pugh - PERSON
Free People - ORG
the Financial Times - ORG
21/10/2024 - DATE


In [ ]:
import spacy
from spacy import displacy

# Load the SpaCy model (ensure it's the correct model for your needs)
nlp = spacy.load("en_core_web_sm")  # Or use a larger model if needed

# Sample text from the article about Elixirr and Hypothesis
text = """
Elixirr, a London, UK-based consulting firm, acquired Hypothesis, a Los Angeles, CA-based insights and strategy firm.
The amount of the deal was not disclosed. The acquisition brings new capabilities to Elixirr’s client base.
Hypothesis’ roster of clients, with a particular focus on the consumer, technology and entertainment industries,
is highly complementary to Elixirr’s established strategy, digital, data, AI and innovation offering.
Led by CEO Maria Vallis, Hypothesis is an insights and strategy agency for brands to innovate, uncover opportunities, and grow.
They help clients understand audiences and attract new customers, position, build, and strengthen brands,
create content and communications, develop products and services that deliver growth,
and operate with purpose and create values-driven culture. Through this acquisition, Vallis will now join Elixirr as Partner,
based out of Los Angeles, California. Founded in 2009, and led by CEO Stephen Newton, Elixirr is a global consulting firm
working with clients across a diverse range of industries, markets and geographies.
It has also acquired seven boutique firms – Den Creative, Coast Digital, The Retearn Group, iOLAP, Responsum, Insigniam
and now Hypothesis – to grow capabilities, expand into new geographies and markets, access new clients and talent, and more.
Elixirr has been quoted on the AIM market of the London Stock Exchange since 2020. FinSMEs 21/10/2024
"""

# Process the text
doc = nlp(text)

# Initialize the data structure for storing classified entities
classified_entities = {
    "Company_Receiving_Investment": None,
    "Investment_Amount": None,
    "Investing_Companies": [],
    "Round_of_Investment": None,
    "Location_of_the_Company": [],
    "Date": None,
    "CEO_Name": None,
}

# Iterate over the detected entities and classify them
for ent in doc.ents:
    if ent.label_ == "ORG":
        if "VCT" in ent.text or "Investments" in ent.text:
            classified_entities["Investing_Companies"].append(ent.text)
        else:
            classified_entities["Company_Receiving_Investment"] = ent.text
    elif ent.label_ == "MONEY":
        classified_entities["Investment_Amount"] = ent.text
    elif ent.label_ == "GPE":
        classified_entities["Location_of_the_Company"].append(ent.text)
    elif ent.label_ == "DATE":
        classified_entities["Date"] = ent.text
    elif ent.label_ == "PERSON":
        if not classified_entities["CEO_Name"]:  # Store only the first name as CEO
            classified_entities["CEO_Name"] = ent.text

# Print the structured data
print(classified_entities)


{'Company_Receiving_Investment': 'the London Stock Exchange', 'Investment_Amount': None, 'Investing_Companies': [], 'Round_of_Investment': None, 'Location_of_the_Company': ['London', 'UK', 'Los Angeles', 'Los Angeles', 'California'], 'Date': '21/10/2024', 'CEO_Name': 'Maria Vallis'}


In [ ]:
# Add your new labels to the NER model
for label in data.columns:
    if 'Start_Pos' not in label and 'End_Pos' not in label and label != 'Article_Text':
        ner.add_label(label)  # Ensure these labels are present in training


In [ ]:
print("Custom labels added to the NER model:", [label for label in data.columns if 'Start_Pos' not in label and 'End_Pos' not in label and label != 'Article_Text'])


Custom labels added to the NER model: ['Article_No', 'Company_Receiving_Investment', 'Investment_Amount', 'Investing_Companies', 'Round_of_Investment', 'Location_of_the_Company', 'Date', 'CEO_Name']


In [ ]:
for iteration in range(10):  # Adjust number of iterations as necessary
    docs = list(doc_bin.get_docs(nlp.vocab))  # Convert generator to list
    random.shuffle(docs)  # Shuffle the training data

    # Create minibatches
    batches = minibatch(docs, size=8)  # Size can be adjusted
    for batch in batches:
        examples = []
        for doc in batch:
            spans = [...]  # Extract spans similarly to how you did during training
            if spans:
                example = Example.from_dict(doc, {"entities": [(span.start, span.end, span.label_) for span in spans]})
                examples.append(example)
                # Debugging: Print examples
                print(f"Training Example: {example}")

        # Update the model
        nlp.update(examples, drop=0.5, losses=losses)

print("Losses during training:", losses)


AttributeError: 'ellipsis' object has no attribute 'start'

In [ ]:
data.columns()

NameError: name 'data' is not defined